In [52]:
###############################################
###############################################
########## Created by James Patchett ##########
################## 11/03/19 ###################
###############################################
###############################################

In [53]:
## Simple programme that uses the drivers in the The Stack 
## to run an FMR experiment

In [54]:
#Add to the path the location of the drivers
import sys
sys.path.append(r'C:\Data\James\Scripts\The Stack')

#Import the drivers
from Instruments.anritsu import anritsu
from Instruments.gaussmeter import gaussmeter_455 as gaussmeter
from Instruments.gen60 import gen60 as gen
from Instruments.lockIn import lockIn
from Instruments.motor import motor

import time

In [55]:
##############################################
############### User Inputs ##################
##############################################

In [56]:
folder_address = r'C:\Data\James\Data\SP75344\FMR\22.03.19\\' #Address to write the data to in CSV format
data_fn = r'results' #File name to write the data to, a new file will be created for each frequency and angle
meta_fn = r'meta' #File name to write the meta-data to

#Short summary of the experiment
experiment_summ = r'Testing out the GEN60 power supply'

## Kepco parameters
#Starting, ending current, and the steps between them (A)
curr_start = 0
curr_end = 5
curr_step = 0.15

## MW source parameters
#Start, end, and step frequency (GHz)
freq_start = 3 #Won't go below 2
freq_end = 30
freq_step = 0.5
MW_power = 13 #dBm

## Pulse parameters
#Frequency of the modulating pulse (Hz)
ref_freq = 421

## Motor parameters
#Start, end, and step angles
angle_start = 0
angle_end = 360
angle_step = 0

## Gradient threshold
#Rate of change of signal at which point it will start to sample the magnetic field more finely
gradient_thres = 20e-6 #V/T
fine_curr_step = curr_step/3

#N.b. if any of the step parameters are set to 0
#The programme will simply skip this loop
#N.b. end parameter is not included in the loop (i.e. [start, end))

In [57]:
###############################################
############ End of user inputs ###############
###############################################

In [58]:
## Set up the drivers for all the instruments

#GPIB0 Addresses and instrument ids
MW_source_add = "GPIB0::23::INSTR"
MW_source_id = "MW source"

gaussmeter_add = "GPIB0::14::INSTR"
gaussmeter_id = "Gaussmeter"

gen_add = "COM6"
gen_id = 'Gen60 power supply'

lock_in_add = "GPIB0::20::INSTR"
lock_in_id = 'Lock-in amplifier'

motor_id = "Motor"

In [59]:
import visa
import matlab.engine

#Set up the resource manager
rm = visa.ResourceManager()

#Check for connectivity by printing the list of connected resources
rm.list_resources()

#Open a matlab engine to connect to the KEPCO
eng = matlab.engine.start_matlab()

#Add the KEPCO and motor functions folder to the engine's path
eng.addpath(r'C:\Data\James\Scripts\The Stack\Instruments\Kepco_func', nargout = 0)
eng.addpath(r'C:\Data\James\Scripts\The Stack\Instruments\Motor_func', nargout = 0)

#Intialise the instruments
MW_source =  anritsu(MW_source_add, rm, MW_source_id)
gssmtr = gaussmeter(gaussmeter_add, rm, gaussmeter_id)
genesis = gen(gen_add, rm, gen_id)
lck_in = lockIn(lock_in_add, rm, lock_in_id)
mtr = motor(eng, motor_id)

Intailising MW source status variables


In [60]:
## Set-up variables
## User can edit them, but the recommended settings are commented in ()

lck_in_ch = 0 #(0); XY, 1; Rtheta
lck_in_tc = 10 #Time-const, 1s (10)
lck_in_fs = 3 #Lock in filter (24 dBm/oct (3))
lck_in_rf_src = 0 #(0) External source

gssmtr_units = 2 #(2) tesla

sleep_time = 10 #Wait (10) seconds between each measurement

In [61]:
## Set up the microwave source
#Set the modulating pulse to the external source
MW_source.set_pulse_sour('EXTernal2')
#Set the pulse modulation on
MW_source.set_pulse_on(1)
#Set the output power
MW_source.set_power(MW_power)


## Set up the lock-in
#Make display 1 display X
lck_in.set_ch_1(lck_in_ch)
#Set the time-const to 1s- N.b. takes around 5 times
#the TC value for the output to settle (i.e. 1.5s)
lck_in.set_time_const(lck_in_tc)
#Set the filter slope ro 24dBm/oct
lck_in.set_filter_slope(lck_in_fs)
#Set the expand to(10x) for the X channel
lck_in.set_expand(1, 1)
#Set to an external source
lck_in.set_rf_src(lck_in_rf_src)
#Set the sensitivity to (16) 500uV/pA
lck_in.set_sens(16)

##Set up the Gaussmeter
#Set the units to Tesla
gssmtr.set_units(gssmtr_units)
#Set auto-range on
gssmtr.set_auto(1)

##Set up the gen60
genesis.set_ovp(55)
time.sleep(1)
genesis.set_output_volt(50) #0A and 50V so its in current mode
genesis.set_output_curr(curr_start)

In [62]:
## Write the user inputs to a meta-file
import csv 

#Ensure the freq/angle metadata makes sense
if(angle_step == 0):
    angle_end = angle_start
    
if(freq_step == 0):
    freq_end = freq_start    

with open(folder_address+meta_fn+'.csv', mode = 'w', newline = '') as csv_file:
    #Open the csv reader
    csv_reader = csv.writer(csv_file, delimiter = ',', quotechar = '"',\
                 quoting = csv.QUOTE_MINIMAL)
    
    #Write the experiment summary
    csv_reader.writerow([experiment_summ])
    
    
    #Write the headers
    csv_reader.writerow(['Start current','End current', 'Current step',\
                       'Start frequency', 'End frequency', 'Frequency step', 'Power'\
                       'Reference frequency','Start angle', 'End angle', 'Angle step'])
    
    #Write the meta-data
    csv_reader.writerow([curr_start, curr_end, curr_step, freq_start, freq_end, freq_step,\
                        MW_power, ref_freq, angle_start, angle_end, angle_step])
       

In [63]:
#Increase the angle/frequency step size to 1 to ensure it plays nice 
#with arange
if(angle_step == 0):
    angle_end = angle_start+1
    angle_step = 1
    
if(freq_step == 0):
    freq_step = 1
    freq_end = freq_start+1
    

In [64]:
####################################
####################################
### Start running the experiment ###
####################################
####################################

In [ ]:
import numpy as np

#Turn the MW output on
MW_source.set_output_on(1)
MW_source.set_freq(float(freq_start))

#Are we doing the negative run?
switched = 1

#Move the motor to the starting angle
current_angle = mtr.getPos()

if(current_angle > 180):
    rel_angle = angle_start+360-current_angle
else:
    rel_angle = angle_start-current_angle
    
mtr.move_Rel(float(rel_angle))

####################################
#### Saturated the magnet along ####
###### The initial direction #######
####################################

genesis.set_output_curr(10)
genesis.set_on(1)
time.sleep(10)
genesis.set_output_curr(curr_start)

## Update the lock-in
time.sleep(30)
lck_in.auto_phase()
lck_in.auto_off(1) #Auto-offset X at the beginning of the measurement
lck_in.auto_reserve() #Probably not that important, but update the dynamic reserve

for angle in np.arange(angle_start, angle_end, angle_step):
    
    print("Angle: "+str(angle))

    for frequency in np.arange(freq_start, freq_end, freq_step):
    
        ## Update the microwave source frequency 
        MW_source.set_freq(float(frequency))
        print("Frequency: "+str(frequency))
        
        #Open the data csv file to write to
        address = folder_address+data_fn+'_'+str(angle)+'deg_'+\
                  str(frequency)+'GHz.csv'

        with open(address, mode = 'w+', newline ='') as data_file:

            data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                         quoting = csv.QUOTE_MINIMAL)

            #Writer the headers
            data_writer.writerow(['Current \\A', 'Magnetic Field \\T', 'Output voltage \\V',\
                                 'Angle \\deg', 'Frequency \\GHz'])
        data_file.close()

        for i in range(2): #Rotate by 180 degrees at the end to sweep the full range of magnetic field
            
            if i == 1:
                switched = -1
                
                ## Slowly ramp down the magnetic field to 0
                for current in np.arange(curr_end, curr_start, 5*curr_step):
                    genesis.set_output_curr(float(current))
                    time.sleep(1)
                
                #Now rotate the sample
                mtr.move_Rel(float(180))

            else:
                switched = 1
                
                #Saturate the magnet along the field direction then slowly ramp down
                for current in np.arange(curr_end, curr_start, 5*curr_step):
                    genesis.set_output_curr(float(current))
                    time.sleep(1)
                                
            
            #Loop over the magentic field values
            for current in np.arange(curr_start, curr_end, curr_step):

                #Change the current
                genesis.set_output_curr(float(current))
                print("Current: "+str(float(current)))

                #Wait between measurements
                if(current == curr_start):
                    time.sleep(3*sleep_time)
                else:
                    time.sleep(sleep_time)

                #List to store the results in
                volt_output = [0, 0, 0]

                #Read the output from the lock-in, 3 times and take the average
                for i in range(3):
                    volt_output[i] = float(lck_in.read(1))

                    #Wait (4) seconds so the results are approximately independent
                    if(i is not 3):
                        time.sleep(4)

                #Calculate the average signal
                ave_output = np.mean(volt_output)

                print("Output: "+str(ave_output))

                #Read the magnetic field from the gaussmeter
                magnetic_field = gssmtr.read()*switched

                #Write the data to the CSV file
                with open(address, mode = 'a', newline ='') as data_file:

                    data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                                             quoting = csv.QUOTE_MINIMAL)

                    data_writer.writerow([current, magnetic_field, ave_output, angle, frequency])

                data_file.close()

                #Estimate the gradient of the voltage signal
                #if it exceeds the threshold value, complete a finer currnet loop
                #Or if the current is sufficently close to zero
                if(current != curr_start and (abs((ave_output-previous_output)/(magnetic_field-previous_field)) > gradient_thres or abs(current) <= 1.5*curr_step)):
                    print("Running fine loop")

                    for fine_current in np.arange(current-curr_step+fine_curr_step/2, current, fine_curr_step):

                        genesis.set_output_curr(float(fine_current))
                        print("Current: "+str(float(fine_current)))

                        #Wait between measurements
                        time.sleep(sleep_time)

                        #List to store the results in
                        volt_output = [0, 0, 0]

                        #Read the output from the lock-in, 3 times and take the average
                        for i in range(3):
                            volt_output[i] = float(lck_in.read(1))

                            #Wait (4) seconds so the results are approximately independent
                            if(i is not 3):
                                time.sleep(4)

                        #Calculate the average signal
                        fine_ave_output = np.mean(volt_output) 

                        #Read the magnetic field from the gaussmeter
                        fine_magnetic_field = gssmtr.read()*switched

                        #Write the data to the CSV file
                        with open(address, mode = 'a', newline ='') as data_file:

                            data_writer = csv.writer(data_file, delimiter = ',', quotechar = '"',\
                                          quoting = csv.QUOTE_MINIMAL)

                            data_writer.writerow([fine_current, fine_magnetic_field, fine_ave_output, angle, frequency])

                        data_file.close()

                #Store this loops values in memory    
                previous_output = ave_output
                previous_field = magnetic_field
            
            #Move it around by 180degrees to do the negative current
            if i == 1:
                mtr.move_Rel(float(180))
                
    
    ## Update the motor angle
    mtr.move_Rel(float(angle_step))
    
##Reset and switch off all the machines outputting power
genesis.reset()
MW_source.reset()

Angle: 0
Frequency: 3.0
Current: 0.0
Output: 3.0423399999999996e-08
Current: 0.15
Output: 5.7121433333333334e-08
Running fine loop
Current: 0.024999999999999998
Current: 0.075
Current: 0.125
Current: 0.3
Output: 1.00583e-07
Current: 0.44999999999999996
Output: 1.6018799999999998e-07
Current: 0.6
Output: 2.68223e-07
Running fine loop
Current: 0.475
Current: 0.525
Current: 0.5750000000000001
Current: 0.75
Output: 4.2841033333333335e-07
Running fine loop
Current: 0.625
Current: 0.675
Current: 0.7250000000000001
Current: 0.8999999999999999
Output: 6.674516666666667e-07
Running fine loop
Current: 0.7749999999999999
Current: 0.825
Current: 0.875
Current: 1.05
